In [1]:
#import sections something to cover all bases
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from patsy import dmatrices
import matplotlib.patches as mpatches
import statsmodels.formula.api as sm
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from statsmodels.formula.api import logit
from sklearn.cross_validation import train_test_split

import os
import glob

import time
import multiprocessing
from multiprocessing import Pool
import threading

import cleaning_functions

C:\Users\Andy\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
# %%time
# pool = Pool(processes=2) # initialize the Pool.
# months=["Jan", "Nov"]
# result = pool.map(cleaning_functions.re_construct, months)       # map f to the data using the Pool of processes to do the work 
# pool.close() # No more processes
# pool.join()
# print("Result: ", result)

In [ ]:
# %%time
# pool = Pool(processes=2) # initialize the Pool.
# months=["Jan", "Nov"]
# result = pool.map(cleaning_functions.complete_extraction, months)       # map f to the data using the Pool of processes to do the work 
# pool.close() # No more processes
# pool.join()
# print("Result: ", result)

In [ ]:
def make_speed_check():
    sequence_dataframe = pd.read_csv("route_seq.csv",
                                     encoding = "latin1",
                                     header = 0,
                                     index_col = None,
                                     converters = {"LineID":str,
                                                   "Stop_ID":str,
                                                   "Stop_Sequence": str})
    
    
    sequence_dataframe = sequence_dataframe[["LineID",
                                             "Stop_ID",
                                             "Stop_Sequence",
                                             "Destination"]]

    LIDS = set( sequence_dataframe.LineID.unique())

    for LID in LIDS:

        DIRS = set (sequence_dataframe[sequence_dataframe["LineID"] == LID].Destination.unique())
        for DIR in DIRS:
            SIDS = set (sequence_dataframe[(sequence_dataframe["LineID"] == LID) &\
                                           (sequence_dataframe["Destination"]==DIR)].Stop_ID.unique())
            for SID in SIDS:
                SEQS = set(sequence_dataframe[(sequence_dataframe["LineID"]==LID) &\
                                              (sequence_dataframe["Destination"]==DIR) &\
                                              (sequence_dataframe["Stop_ID"] == SID)].Stop_Sequence.unique())
                if len(SEQS) != 1:
                    print("Problem:", LID,":", DIR, SID, SEQS)


    


In [ ]:
make_speed_check()



Only the FINAL outputs appear as cell output, but you can track progress in the ipy terminal. (if you activate jupyter notebook withing terminal you will see outputs such as print statements running in each function.) When multiprocessing  you can expect 4 outputs to occur simultaneously. 

e.g.
reconstucting x
reconstucting y
reconstucting a
reconstucting b



In [ ]:
def join_months(iterator):
    print("joining routes : ")
    
    
    path = os.path.join(os.getcwd(), "NovJan_routes")
    os.makedirs(path, exist_ok=True)
    
    Nov_path = os.path.join(os.getcwd(), "Nov_routes")
    Jan_path = os.path.join(os.getcwd(), "Jan_routes")
    contents = os.listdir(Nov_path)
    
    
    for file in iterator:
        if file != '.DS_Store':
            Nov_path_to_file = os.path.join(Nov_path, file)
            Nov_dataframe = pd.read_hdf(Nov_path_to_file)
            
            Jan_path_to_file = os.path.join(Jan_path, file)
            Jan_dataframe = pd.read_hdf(Jan_path_to_file)
            
            Jan_dataframe, Nov_dataframe = Jan_dataframe.align(Nov_dataframe, axis=1)
            Combined_dataframe = pd.concat([Jan_dataframe,Nov_dataframe])
            
        
            write_address = os.path.join(path,'x' + file)
            Combined_dataframe.to_hdf(write_address, key="moo", mode="w")

    return 'success'

In [9]:
%%time
pool = Pool(processes=2) # initialize the Pool.
Nov_path = os.path.join(os.getcwd(),"Nov_routes")
contents = os.listdir(Nov_path)
total = len(contents)
iterators=[contents[:total//2],
           contents[total//2:]
          ]

result = pool.map(cleaning_functions.join_months, iterators )       # map f to the data using the Pool of processes to do the work 
pool.close() # No more processes
pool.join()
print("Result: ", result)

Result:  ['success']
Wall time: 10.9 s


In [ ]:
#open data
